In [50]:
import pandas as pd
import numpy as np
import soccerdata as sd 
import requests
import json
import itertools
import mezzala
import fotmob_scraper as fms
import pandas_montecarlo

In [51]:
mh = pd.read_csv('https://www.football-data.co.uk/new/SWZ.csv')

In [52]:
mh['Home'].unique()

array(['Servette', 'Thun', 'Grasshoppers', 'Luzern', 'St. Gallen',
       'Basel', 'Lausanne', 'Sion', 'Young Boys', 'Zurich', 'Aarau',
       'Vaduz', 'Lugano', 'Xamax', 'Winterthur'], dtype=object)

In [53]:
url = 'https://raw.githubusercontent.com/openfootball/football.json/master/2020-21/ch.1.json'


In [54]:
from urllib import response


response = requests.get(url).json()

data = response['matches']
data[0:3]

[{'round': 'Spieltag 1',
  'date': '2020-09-19',
  'team1': 'FC Lugano',
  'team2': 'FC Luzern',
  'score': {'ft': [2, 1]}},
 {'round': 'Spieltag 1',
  'date': '2020-09-19',
  'team1': 'BSC Young Boys',
  'team2': 'FC Zürich',
  'score': {'ft': [2, 1]}},
 {'round': 'Spieltag 1',
  'date': '2020-09-20',
  'team1': 'FC Basel',
  'team2': 'FC Vaduz',
  'score': {'ft': [2, 2]}}]

In [55]:
mh

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Switzerland,Super League,2012/2013,13/07/2012,18:45,Servette,Basel,0,1,A,6.22,4.17,1.60,6.22,4.20,1.68,5.32,3.85,1.59
1,Switzerland,Super League,2012/2013,14/07/2012,18:45,Thun,Lausanne,0,0,D,1.75,3.80,5.23,1.77,3.81,5.38,1.70,3.51,4.85
2,Switzerland,Super League,2012/2013,15/07/2012,12:45,Grasshoppers,Sion,0,2,A,3.27,3.36,2.39,3.35,3.41,2.41,3.07,3.24,2.25
3,Switzerland,Super League,2012/2013,15/07/2012,12:45,Luzern,Zurich,1,1,D,2.04,3.65,3.89,2.20,3.80,3.89,2.10,3.33,3.31
4,Switzerland,Super League,2012/2013,15/07/2012,15:00,St. Gallen,Young Boys,1,1,D,3.62,3.67,2.12,3.69,3.67,2.15,3.32,3.43,2.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1806,Switzerland,Super League,2021/2022,29/05/2022,15:30,Luzern,Schaffhausen,2,0,H,1.72,4.29,4.49,1.73,4.61,4.79,1.69,4.11,4.29
1807,Switzerland,Super League,2022/2023,16/07/2022,17:00,Young Boys,Zurich,4,0,H,1.65,4.41,4.90,1.76,4.57,5.20,1.63,4.26,4.63
1808,Switzerland,Super League,2022/2023,16/07/2022,19:30,Winterthur,Basel,1,1,D,3.05,3.57,2.35,3.10,3.76,2.45,2.88,3.50,2.30
1809,Switzerland,Super League,2022/2023,17/07/2022,15:30,Lugano,Sion,2,3,A,1.85,3.75,4.36,1.95,3.81,4.36,1.87,3.63,3.89


In [56]:
mh_dict = mh.to_dict('records')

In [57]:
mh_dict[0]

{'Country': 'Switzerland',
 'League': 'Super League',
 'Season': '2012/2013',
 'Date': '13/07/2012',
 'Time': '18:45',
 'Home': 'Servette',
 'Away': 'Basel',
 'HG': 0,
 'AG': 1,
 'Res': 'A',
 'PH': 6.22,
 'PD': 4.17,
 'PA': 1.6,
 'MaxH': 6.22,
 'MaxD': 4.2,
 'MaxA': 1.68,
 'AvgH': 5.32,
 'AvgD': 3.85,
 'AvgA': 1.59}

In [58]:
adapter = mezzala.KeyAdapter(
    home_team='Home',
    away_team='Away',
    home_goals='HG',
    away_goals='AG'
)

In [59]:
model = mezzala.DixonColes(adapter,
blocks=[
    mezzala.blocks.HomeAdvantage(),
    mezzala.blocks.TeamStrength(),
    mezzala.blocks.BaseRate(),
])
model.fit(mh_dict)

DixonColes(adapter=KeyAdapter(home_goals='HG', away_goals='AG', home_team='Home', away_team='Away'), blocks=[TeamStrength(), HomeAdvantage(), BaseRate()]), weight=UniformWeight()

In [60]:
match_to_predict = {
    'Home': 'Basel',
    'Away': 'Wintherthur'
}

scorelines = model.predict_one(match_to_predict)

In [61]:
scorelines

[ScorelinePrediction(home_goals=0, away_goals=0, probability=0.05029659937559772),
 ScorelinePrediction(home_goals=0, away_goals=1, probability=0.03542797671575925),
 ScorelinePrediction(home_goals=0, away_goals=2, probability=0.018174262529898153),
 ScorelinePrediction(home_goals=0, away_goals=3, probability=0.005432989442143524),
 ScorelinePrediction(home_goals=0, away_goals=4, probability=0.0012180978827841505),
 ScorelinePrediction(home_goals=0, away_goals=5, probability=0.00021848191944327827),
 ScorelinePrediction(home_goals=0, away_goals=6, probability=3.265634464347192e-05),
 ScorelinePrediction(home_goals=0, away_goals=7, probability=4.1838186291138e-06),
 ScorelinePrediction(home_goals=0, away_goals=8, probability=4.690144043484198e-07),
 ScorelinePrediction(home_goals=0, away_goals=9, probability=4.6735510411060356e-08),
 ScorelinePrediction(home_goals=0, away_goals=10, probability=4.191315067977678e-09),
 ScorelinePrediction(home_goals=0, away_goals=11, probability=3.417125

In [62]:
mh['Home'].unique

<bound method Series.unique of 0           Servette
1               Thun
2       Grasshoppers
3             Luzern
4         St. Gallen
            ...     
1806          Luzern
1807      Young Boys
1808      Winterthur
1809          Lugano
1810        Servette
Name: Home, Length: 1811, dtype: object>

In [63]:
mezzala.scorelines_to_outcomes(scorelines)

{Outcomes('Home win'): OutcomePrediction(outcome=Outcomes('Home win'), probability=0.6652401934955264),
 Outcomes('Draw'): OutcomePrediction(outcome=Outcomes('Draw'), probability=0.19947600468080312),
 Outcomes('Away win'): OutcomePrediction(outcome=Outcomes('Away win'), probability=0.1352838018236712)}

In [64]:
matches_to_predict = [{
    'Home': 'Basel',
    'Away': 'Zurich'
},{
    'Home': 'Zurich',
    'Away': 'Basel'
},]
scorelines = model.predict(matches_to_predict)

In [65]:
scorelines

[[ScorelinePrediction(home_goals=0, away_goals=0, probability=0.04304064857162428),
  ScorelinePrediction(home_goals=0, away_goals=1, probability=0.03387900630911706),
  ScorelinePrediction(home_goals=0, away_goals=2, probability=0.019867072807889723),
  ScorelinePrediction(home_goals=0, away_goals=3, probability=0.006768111112287078),
  ScorelinePrediction(home_goals=0, away_goals=4, probability=0.00172926813896582),
  ScorelinePrediction(home_goals=0, away_goals=5, probability=0.0003534656268882446),
  ScorelinePrediction(home_goals=0, away_goals=6, probability=6.020752680293709e-05),
  ScorelinePrediction(home_goals=0, away_goals=7, probability=8.790378973978691e-06),
  ScorelinePrediction(home_goals=0, away_goals=8, probability=1.1229811417797251e-06),
  ScorelinePrediction(home_goals=0, away_goals=9, probability=1.2752190887342446e-07),
  ScorelinePrediction(home_goals=0, away_goals=10, probability=1.3032857787135086e-08),
  ScorelinePrediction(home_goals=0, away_goals=11, probabi

In [66]:
final_score = scorelines[0][0]
for scores in scorelines[0]:
    if scores.probability > final_score.probability:
        final_score = scores

In [67]:
final_score.probability

0.09815550356210534

In [68]:
scorelines[0][0]

ScorelinePrediction(home_goals=0, away_goals=0, probability=0.04304064857162428)

In [69]:
df_schedule = fms.get_league_schedule(69)

getting fixtures for Super League


In [70]:
df_schedule

['id',
 'pageUrl',
 'displayTournament',
 'lnameArr',
 'color',
 'notStarted',
 'opponent.id',
 'opponent.name',
 'opponent.score',
 'home.id',
 'home.name',
 'home.score',
 'away.id',
 'away.name',
 'away.score',
 'status.finished',
 'status.started',
 'status.cancelled',
 'status.scoreStr',
 'status.startDateStr',
 'status.startDateStrShort',
 'status.reason.short',
 'status.reason.long',
 'status.startTimeStr']

In [71]:
league_url = "https://www.fotmob.com/api/leagues?id={}"
league_id = 69
response = requests.get(league_url.format(league_id)).json()
df_fixtures = pd.json_normalize(response, record_path=["matches"])

In [72]:
df_fixtures_short = df_fixtures[['id','home.name','away.name', 'home.id', 'away.id','status.startDateStr']]

In [73]:
df_fixtures_short['home.name'].unique()

array(['Young Boys', 'Winterthur', 'Lugano', 'Servette', 'FC Zürich',
       'St. Gallen', 'Basel', 'Grasshopper', 'Sion', 'Luzern'],
      dtype=object)

In [74]:
mh["Home"].unique()

array(['Servette', 'Thun', 'Grasshoppers', 'Luzern', 'St. Gallen',
       'Basel', 'Lausanne', 'Sion', 'Young Boys', 'Zurich', 'Aarau',
       'Vaduz', 'Lugano', 'Xamax', 'Winterthur'], dtype=object)

In [75]:
dict_teams = {
    'Young Boys':'Young Boys', 
    'Thun':'Winterthur', 
    'Lugano':'Lugano', 
    'Servette':'Servette', 
    'FC Zürich':'Zurich',
    'St. Gallen':'St. Gallen', 
    'Basel':'Basel', 
    'Grasshoppers':'Grasshopper', 
    'Sion':'Sion', 
    'Luzern':'Luzern',
    'Lausanne':'Winterthur',
    'Aarau':'Winterthur',
    'Vaduz':'Winterthur',
    'Xamax':'Winterthur'
}

In [76]:
mh['Home'] = mh['Home'].map(dict_teams)
mh['Away'] = mh['Away'].map(dict_teams)

In [77]:
mh = mh[mh['Home'] != mh['Away']]

In [78]:
df_fixtures_short

,id,home.name,away.name,home.id,away.id,status.startDateStr
0,3905393,Young Boys,FC Zürich,10192,10243,"Sat, 16 Jul"
1,3905401,Winterthur,Basel,7894,9931,"Sat, 16 Jul"
2,3905400,Lugano,Sion,7896,10179,"Sun, 17 Jul"
3,3905399,Servette,St. Gallen,9777,10190,"Sun, 17 Jul"
4,3905397,FC Zürich,Luzern,10243,10199,Today
...,...,...,...,...,...,...
175,3905541,Basel,Grasshopper,9931,9956,29 May 2023
176,3905573,FC Zürich,Lugano,10243,7896,29 May 2023
177,3905540,Luzern,Servette,10199,9777,29 May 2023
178,3905528,St. Gallen,Sion,10190,10179,29 May 2023


In [79]:
mh_dict = mh.to_dict('records')

In [80]:
season_end_date = '2020-06-30'

model = mezzala.DixonColes(adapter,
blocks=[
    mezzala.blocks.HomeAdvantage(),
    mezzala.blocks.TeamStrength(),
    mezzala.blocks.BaseRate(),
]
)
model.fit(mh_dict)

DixonColes(adapter=KeyAdapter(home_goals='HG', away_goals='AG', home_team='Home', away_team='Away'), blocks=[TeamStrength(), HomeAdvantage(), BaseRate()]), weight=UniformWeight()

In [81]:
df_predictions = pd.DataFrame()
df_score_pred = pd.DataFrame()

for index, row in df_fixtures_short.iterrows():
    _match_to_predict = {
        'Home': row['home.name'],
        'Away': row['away.name']
    }

    _scorelines = model.predict_one(_match_to_predict)

    _pred = mezzala.scorelines_to_outcomes(_scorelines)

    _df_scores = pd.DataFrame()

    for scoreline in _scorelines:
        _dict_scorelines = {
            'home': [row['home.name']],
            'away': [row['away.name']], 
            'game_id': [row['id']],
            'date': [row['status.startDateStr']],
            'home_goals': [scoreline.home_goals],
            'away_goals': [scoreline.away_goals],
            'prob':[scoreline.probability]
        }
        _df_scores = pd.concat([_df_scores,pd.DataFrame(_dict_scorelines)], ignore_index=True, axis=0)
    _df_scores['home_score_pred'] = _df_scores['home_goals'] * _df_scores['prob']
    _df_scores['away_score_pred'] = _df_scores['away_goals'] * _df_scores['prob']
    _df_scores['home_most_probable_score'] = _df_scores.sort_values('prob',ascending=False).head(1)['home_goals'].values[0]
    _df_scores['away_most_probable_score'] = _df_scores.sort_values('prob',ascending=False).head(1)['away_goals'].values[0]
    df_score_pred = pd.concat([df_score_pred, _df_scores.groupby(['home','away','game_id','date']).sum().reset_index()], ignore_index=True, axis=0)

    



    
    _dict_pred = {
        'home': [row['home.name']],
        'away': [row['away.name']], 
        'game_id': [row['id']],
        'date': [row['status.startDateStr']],
        'prob_away': [_pred.popitem()[1].probability],
        'prob_draw': [_pred.popitem()[1].probability],
        'prob_home': [_pred.popitem()[1].probability],}
    _pred_series = pd.DataFrame(_dict_pred)

    df_predictions = pd.concat([df_predictions, _pred_series], ignore_index=True, axis=0)

df_score_pred['home_score_pred_int'] = df_score_pred['home_score_pred'].round(0).astype(int)
df_score_pred['away_score_pred_int'] = df_score_pred['away_score_pred'].round(0).astype(int)

In [82]:
df_predictions.head()

,home,away,game_id,date,prob_away,prob_draw,prob_home
0,Young Boys,FC Zürich,3905393,"Sat, 16 Jul",0.185832,0.211550,0.602618
1,Winterthur,Basel,3905401,"Sat, 16 Jul",0.592667,0.222548,0.184785
2,Lugano,Sion,3905400,"Sun, 17 Jul",0.308969,0.273197,0.417834
3,Servette,St. Gallen,3905399,"Sun, 17 Jul",0.343018,0.259926,0.397057
4,FC Zürich,Luzern,3905397,Today,0.276099,0.249590,0.474311


In [83]:
df_predictions['points_home'] = 3 * df_predictions['prob_home'] + 1 * df_predictions['prob_draw']
df_predictions['points_away'] = 3 * df_predictions['prob_away'] + 1 * df_predictions['prob_draw']

In [84]:
df_points = pd.concat([df_predictions[['home','points_home']].groupby('home').sum(),df_predictions[['away','points_away']].groupby('away').sum()], axis=1)
df_points['points_total'] = df_points['points_home'] + df_points['points_away']

In [85]:
df_points.sort_values(by='points_total', ascending=False)

,points_home,points_away,points_total
Basel,38.769447,31.643589,70.413036
Young Boys,37.537399,30.127848,67.665247
FC Zürich,29.004313,21.656205,50.660518
Luzern,27.337191,19.987789,47.324980
Grasshopper,27.156669,19.934074,47.090743
St. Gallen,26.782772,19.419959,46.202731
Sion,25.803841,18.743652,44.547492
Lugano,25.072773,18.112035,43.184807
Servette,24.154030,17.200040,41.354070
Winterthur,22.280754,15.504201,37.784955


In [93]:
df_score_pred['away_most_probable_score'] = df_score_pred['away_most_probable_score'] / 676
df_score_pred['home_most_probable_score'] = df_score_pred['home_most_probable_score'] / 676

In [94]:
df_score_pred.head(15)

,home,away,game_id,date,home_goals,away_goals,prob,home_score_pred,away_score_pred,home_most_probable_score,away_most_probable_score,home_score_pred_int,away_score_pred_int
0,Young Boys,FC Zürich,3905393,"Sat, 16 Jul",8450,8450,1.0,2.147906,1.124016,2.0,1.0,2,1
1,Winterthur,Basel,3905401,"Sat, 16 Jul",8450,8450,1.0,1.036829,2.003955,1.0,1.0,1,2
2,Lugano,Sion,3905400,"Sun, 17 Jul",8450,8450,1.0,1.428718,1.194919,1.0,1.0,1,1
3,Servette,St. Gallen,3905399,"Sun, 17 Jul",8450,8450,1.0,1.503398,1.382569,1.0,1.0,2,1
4,FC Zürich,Luzern,3905397,Today,8450,8450,1.0,1.705698,1.254144,1.0,1.0,2,1
5,St. Gallen,Winterthur,3905396,Today,8450,8450,1.0,1.814869,1.155509,1.0,1.0,2,1
6,Basel,Servette,3905395,Tomorrow,8450,8450,1.0,2.355997,0.874102,2.0,0.0,2,1
7,Grasshopper,Lugano,3905394,Tomorrow,8450,8450,1.0,1.574639,1.138131,1.0,1.0,2,1
8,Sion,Young Boys,3905402,Tomorrow,8450,8450,1.0,1.238462,1.808325,1.0,1.0,1,2
9,St. Gallen,FC Zürich,3905403,"Sat, 30 Jul",8450,8450,1.0,1.556713,1.414166,1.0,1.0,2,1


In [95]:
_df_scores.shape

(676, 11)

In [96]:
_df_scores.head(15)

,home,away,game_id,date,home_goals,away_goals,prob,home_score_pred,away_score_pred,home_most_probable_score,away_most_probable_score
0,Young Boys,Winterthur,3905572,29 May 2023,0,0,3.650627e-02,0.0,0.000000e+00,2,0
1,Young Boys,Winterthur,3905572,29 May 2023,0,1,2.609160e-02,0.0,2.609160e-02,2,0
2,Young Boys,Winterthur,3905572,29 May 2023,0,2,1.376178e-02,0.0,2.752355e-02,2,0
3,Young Boys,Winterthur,3905572,29 May 2023,0,3,4.213068e-03,0.0,1.263920e-02,2,0
4,Young Boys,Winterthur,3905572,29 May 2023,0,4,9.673503e-04,0.0,3.869401e-03,2,0
5,Young Boys,Winterthur,3905572,29 May 2023,0,5,1.776884e-04,0.0,8.884419e-04,2,0
6,Young Boys,Winterthur,3905572,29 May 2023,0,6,2.719900e-05,0.0,1.631940e-04,2,0
7,Young Boys,Winterthur,3905572,29 May 2023,0,7,3.568619e-06,0.0,2.498034e-05,2,0
8,Young Boys,Winterthur,3905572,29 May 2023,0,8,4.096901e-07,0.0,3.277521e-06,2,0
9,Young Boys,Winterthur,3905572,29 May 2023,0,9,4.180789e-08,0.0,3.762710e-07,2,0


In [97]:
_df_scores.sort_values('prob',ascending=False).head(1)['home_goals'].values[0]
_df_scores.sort_values('prob',ascending=False).head(1)['away_goals'].values[0]

0